In [1]:
import keras
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import GRU, LSTM, Dense, SimpleRNN, Dropout, Embedding
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix as cm
from sklearn.model_selection import train_test_split
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../label_news.csv')[['post_time','title', 'content']]
df = np.array(df)
time = df[:,0]
date = []
for i in range(time.shape[0]):
    tmp = datetime.strptime(time[i][2:-9], '%y/%m/%d')
    date.append((tmp.month, tmp.day))
date=np.array(date)
del_row = np.load('../del_row.npy', allow_pickle=True)
del_nodata = np.load('../del_nodata.npy', allow_pickle=True)
date = np.delete(date, del_row, axis=0)
date = np.delete(date, del_nodata, axis=0)
print(date.shape)

(125058, 2)


In [3]:
data = np.load('train.npy', allow_pickle = True)[:,0]
label = np.load('train.npy', allow_pickle=True)[:,3]

In [4]:
print(date.shape)
print(data.shape)
print(label.shape)

(125058, 2)
(125058,)
(125058,)


In [5]:
#w2v_model = Word2Vec(data,min_count = 10, size=250, sg = 1, iter = 10)
#w2v_model.save('q3.model')
w2v_model = Word2Vec.load("q3.model")
embedding_matrix = np.zeros((len(w2v_model.wv.vocab.items()) + 1, w2v_model.vector_size))
word2idx = {}

vocab_list = [(word, w2v_model.wv[word]) for word, _ in w2v_model.wv.vocab.items()]
for i, vocab in enumerate(vocab_list):
    word, vec = vocab
    embedding_matrix[i + 1] = vec
    word2idx[word] = i + 1

embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1],
                            mask_zero=True,
                            weights=[embedding_matrix],
                            trainable=True)

def text_to_index(corpus):
    new_corpus = []
    for doc in corpus:
        new_doc = []
        for word in doc:
            try:
                new_doc.append(word2idx[word])
            except:
                new_doc.append(0)
        new_corpus.append(new_doc)
    return np.array(new_corpus)
def rnn_model():
    model = Sequential()
    model.add(embedding_layer)
    model.add(GRU(16,activation='tanh', return_sequences = True))
    model.add(Dropout(0.3))
    model.add(GRU(16))
    model.add(Dropout(0.3))
    model.add(Dense(32))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(Dense(128))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

### shift-window model training

這裡的準確率指的是整個output跟label的準確率，並非由投票結果

In [21]:
non_predict = []
acc = []
save = 0
accept = [1,2,3,4]
for year in range(2016,2018):
    for mon in range(12):
        change = 0
        if 12 not in accept[:-1]:
            print('\n訓練資料(年/月): %d/%d, %d/%d, %d/%d'%(year,accept[0],year,accept[1],year,accept[2]))
            print('預測資料(年/月): %d/%d'%(year, accept[-1]))
        else:
            change = 1
        x_train = []
        x_test = []

        y_train = []
        y_test = []

        testDate = []
        print('collect data...')
        b = 1
        for i in range(save,data.shape[0]):
            if date[i,0] == accept[0]:
                x_train.append(data[i])
                y_train.append(label[i])
            elif date[i,0] in accept[1:-1]:
                if b:
                    save = i
                    b = 0
                x_train.append(data[i])
                y_train.append(label[i])
            elif date[i,0] == accept[-1]:
                x_test.append(data[i])
                y_test.append(label[i])
                testDate.append(date[i])
            else:
                for x in range(4):
                    accept[x] += 1
                    accept[x] %= 12
                    if accept[x] == 0:
                        accept[x] = 12
                break
        if not change:
            x_train = np.array(x_train)
            y_train = np.array(y_train)
            x_test = np.array(x_test)
            y_test = np.array(y_test)

            PADDING_LENGTH = 100
            x_train = text_to_index(x_train)
            x_train = pad_sequences(x_train, maxlen=PADDING_LENGTH)
            x_test = text_to_index(x_test)
            x_test = pad_sequences(x_test, maxlen=PADDING_LENGTH)


            model = rnn_model()

            model.compile(optimizer='adam',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])

            #print(x_train.shape, x_test.shape)
            #print(sum(y_train)/y_train.shape)
            #model.summary()
            model.fit(x=x_train, y=y_train, batch_size=1000, epochs=10, shuffle = True, class_weight={0:1.5, 1:1}, verbose=0)

            output = model.predict(x_test)
            output = np.array(output)
            output = output.reshape(output.shape[0])

            for i in range(output.shape[0]):
                if output[i] > 0.5:
                    output[i] = 1
                else:
                    output[i] = 0
            output = output.astype('int')
            print('準確率:',(sum(output == y_test.astype('int'))/y_test.shape)[0])

            sameDate = []
            over = 0
            buy = 0
            total = 1
            correct = 0
            skip = 0
            for i in range(x_test.shape[0]):
                if testDate[i][1] != testDate[over][1]:
                    predict = -1
                    total += 1
                    positive = sum(output[over:i] == 1)
                    negative = sum(output[over:i] == 0)
                    if positive > negative and abs(positive - negative)/(i-over) > 0.01:
                        buy += 1
                        predict = 1
                    elif positive < negative and abs(positive - negative)/(i-over) > 0.01:
                        buy += 1
                        predict = 0
                    #print(testDate[i-1][1], predict, y_test[over])
                    if predict == y_test[over]:
                        correct += 1
                    elif predict == -1:
                        skip += 1
                    over = i
            #print('日期準確率:',correct/total)
            #print('不預測率:', skip/total)
            acc.append(correct/total)
            non_predict.append(skip/total)
        else:
            print('skip')
acc = np.array(acc)
non_predict = np.array(non_predict)


訓練資料(年/月): 2016/1, 2016/2, 2016/3
預測資料(年/月): 2016/4
collect data...
準確率: 0.48441926345609065

訓練資料(年/月): 2016/2, 2016/3, 2016/4
預測資料(年/月): 2016/5
collect data...
準確率: 0.6190476190476191

訓練資料(年/月): 2016/3, 2016/4, 2016/5
預測資料(年/月): 2016/6
collect data...
準確率: 0.5555555555555556

訓練資料(年/月): 2016/4, 2016/5, 2016/6
預測資料(年/月): 2016/7
collect data...
準確率: 0.5479857819905213

訓練資料(年/月): 2016/5, 2016/6, 2016/7
預測資料(年/月): 2016/8
collect data...
準確率: 0.5735461801596351

訓練資料(年/月): 2016/6, 2016/7, 2016/8
預測資料(年/月): 2016/9
collect data...
準確率: 0.5330156569094622

訓練資料(年/月): 2016/7, 2016/8, 2016/9
預測資料(年/月): 2016/10
collect data...
準確率: 0.5734177215189873

訓練資料(年/月): 2016/8, 2016/9, 2016/10
預測資料(年/月): 2016/11
collect data...
準確率: 0.5747252747252747

訓練資料(年/月): 2016/9, 2016/10, 2016/11
預測資料(年/月): 2016/12
collect data...
準確率: 0.5
collect data...
skip
collect data...
skip
collect data...
skip

訓練資料(年/月): 2017/1, 2017/2, 2017/3
預測資料(年/月): 2017/4
collect data...
準確率: 0.5574667709147771

訓練資料(年/月): 201

### 結果

這裡的準確率才是按照投票後、一整個月份的準確率

In [22]:
tmp = pd.DataFrame(acc, columns=['按月並投票的準確率'])
tmp['不出手的比例'] = non_predict
idx = []
for i in range(2016,2018):
    for j in range(4,13):
        idx.append(str(i)+'/'+str(j))
tmp.index = idx

In [23]:
tmp.to_csv('./predict_by_month.csv')

In [6]:
tmp = pd.read_csv('./predict_by_month.csv')
tmp

,Unnamed: 0,按月並投票的準確率,不出手的比例
0,2016/4,0.315789,0.000000
1,2016/5,0.809524,0.000000
2,2016/6,0.476190,0.000000
3,2016/7,0.600000,0.000000
4,2016/8,0.782609,0.043478
5,2016/9,0.368421,0.000000
6,2016/10,0.600000,0.000000
7,2016/11,0.727273,0.045455
8,2016/12,0.500000,0.045455
9,2017/4,0.625000,0.000000
